<a href="https://colab.research.google.com/github/dzanahmed/welcome-ideathon-lshtm/blob/main/code/regular_expression_filter_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

## Load packages

In [ ]:
import re
import pandas as pd

## Load data for testing

In [188]:
url = "https://raw.githubusercontent.com/dzanahmed/welcome-ideathon-lshtm/main/data/interim/vax_tweets_v0.csv?token=GHSAT0AAAAAACB5CGEQ4RAN2HFU76LWC2JMZFJ2QXA"
raw_df = pd.read_csv(url)

1000

In [193]:
random_sample = raw_df.sample(frac=0.1, random_state=42)
random_sample = random_sample.loc[:, ['tweet_id', 'text']]
len(random_sample)

10000

## Keyword/RegEx dictionaries

In [170]:
vaccine_filter = [
    r"(?=.*\bCOVID\s+vaccine\b)",
    r"(?=.*\bvaccine\b)",
    r"(?=.*\bvaccination\b)",
    r"(?=.*\bimmune\b)",
    r"(?=.*\bimmunity\b)",
    r"(?=.*\bCOVID\s+vaccination\b)",
    r"(?=.*\bcorona\s+vaccine\b)",
    r"(?=.*\bCOVID19\s+vaccination\b)",
    r"(?=.*\bCOVID-19\s+vaccination\b)",
    r"(?=.*\bcoronavirus\s+vaccination\b)",
    r"(?=.*\bcoronavirus\s+vaccine\b)",
    r"(?=.*\bCOVID-19\s+vaccine\b)",
    r"(?=.*\bModerna\b)",
    r"(?=.*\bPfizer\b)",
    r"(?=.*\bJ&J\b)",
    r"(?=.*\bJohnson\s+&\s+Johnson\b)",
    r"(?=.*\bCOVID\s+vax\b)",
    r"(?=.*\bcorona\s+vax\b)",
    r"(?=.*\bcovid-19\s+vax\b)",
    r"(?=.*\bcovid19\s+vax\b)",
    r"(?=.*\bcoronavirus\s+vax\b)",
    r"(?=.*\bvaccinated\b)",
    r"(?=.*\bcovid\b)",
    r"(?=.*\bvaccinating\b)",
    r"(?=.*\bCovaxin\b)",
    r"(?=.*\bjab\b)"
]

hesitancy_filter = [
    r"(?=.*\b(worry|worries|worried|worrying)\b)",
    r"(?=.*\b((dont|do\s*not)\s*(want|give|take|have))\b)",
    r"(?=.*\b(reluctant)\b)",
    r"(?=.*\bskeptic\w*\b)",
    r"(?=.*\bconflicted\b)",
    r"(?=.*\bdoubt\w*\b)",
    r"(?=.*\bcan\s*not\b)",
    r"(?=.*\bwill\s*not\b)",
    r"(?=.*\bwont\b)",
    r"(?=.*\bunsure\b)",
    r"(?=.*\bunwilling\b)",
    r"(?=.*\bhesitant\b)",
    r"(?=.*\bquestion\w*\b)",
    r"(?=.*\bon\s*the\s*fence\b)",
    r"(?=.*\bwaver\w*\b)",
    r"(?=.*\b(scared|scary)\b)",
    r"(?=.*\bafraid\b)",
    r"(?=.*\bnervous\b)",
    r"(?=.*\bfright\b)",
    r"(?=.*\bhorror\b)"

]

safety_filter = [
    r"(?=.*\bmortality\w*\b)",
    r"(?=.*\bdeath\w*\b)",
    r"(?=.*\bkill\w*\b)",
    r"(?=.*\b(side\s?effects\w*))\b",
    r"(?=.*\bpoison\w*\b)",
    r"(?=.*\blead\w*\b)",
    r"(?=.*\b(worse\s?than\s?infection\w*))\b",
    r"(?=.*\bhazardous\w*\b)",
    r"(?=.*\blethal\w*\b)",
    r"(?=.*\bfatality\w*\b)",
    r"(?=.*\bharmful\w*\b)",
    r"(?=.*\b(adverse\s?reactions\w*))\b",
    r"(?=.*\btoxic\w*\b)",
    r"(?=.*\b(result\s?in\s?death\w*))\b",
    r"(?=.*\bdeadly\w*\b)",
    r"(?=.*\beliminate\w*\b)",
    r"(?=.*\bcomplications\w*\b)",
    r"(?=.*\bdetrimental\w*\b)",
    r"(?=.*\b(potentially\s?lethal\w*))\b",
    r"(?=.*\bperilous\w*\b)",
    r"(?=.*\binjurious\w*\b)",
    r"(?=.*\b(severe\s?consequences\w*))\b",
    r"(?=.*\bhazard\w*\b)",
    r"(?=.*\bdestructive\w*\b)",
    r"(?=.*\bendanger\w*\b)",
    r"(?=.*\bcatastrophic\w*\b)",
    r"(?=.*\b(risks\s?outweigh\s?benefits\w*))\b",
    r"(?=.*\blife-threatening\w*\b)",
    r"(?=.*\bunsafe\w*\b)",
    r"(?=.*\bfatal\w*\b)",
    r"(?=.*\bmortality\w*\b)",
    r"(?=.*\b(negative\s?outcomes\w*))\b",
    r"(?=.*\btoxicity\w*\b)",
    r"(?=.*\bpoisonous\w*\b)",
    r"(?=.*\b(inflict\s?harm\w*))\b",
    r"(?=.*\bcompromising\w*\b)",
    r"(?=.*\bdetriment\w*\b)",
    r"(?=.*\b(critical\s?condition\w*))\b",
    r"(?=.*\brisky\w*\b)",
    r"(?=.*\bincurable\w*\b)",
    r"(?=.*\bmenacing\w*\b)"
]

mistrust_filter = [
    r"(?=.*\b(doubt\w*)\b)",
    r"(?=.*\b(suspic\w*)\b)",
    r"(?=.*\b(skeptic\w*)\b)",
    r"(?=.*\b(distrust\w*)\b)",
    r"(?=.*\b(cynic\w*)\b)",
    r"(?=.*\b(misbelief\w*)\b)",
    r"(?=.*\b(disbelief\w*)\b)",
    r"(?=.*\b(wari\w*)\b)",
    r"(?=.*\b(apprehen\w*)\b)",
    r"(?=.*\b(disillusion\w*)\b)",
    r"(?=.*\b(misgiv\w*)\b)",
    r"(?=.*\b(uncertain\w*)\b)",
    r"(?=.*\b(reservation\w*)\b)",
    r"(?=.*\b(hesitat\w*)\b)",
    r"(?=.*\b(question\w*)\b)",
    r"(?=.*\b(lack\sof\sconfidenc\w*)\b)",
    r"(?=.*\b(unreliab\w*)\b)",
    r"(?=.*\b(dubiousn\w*)\b)",
    r"(?=.*\b(skepti\smindset\w*)\b)",
    r"(?=.*\b(lack\sof\sfaith\w*)\b)",
    r"(?=.*\b(disreputable\w*)\b)",
    r"(?=.*\b(paranoia\w*)\b)",
    r"(?=.*\b(suspiciousness\w*)\b)",
    r"(?=.*\b(doubtfulness\w*)\b)",
    r"(?=.*\b(mistrust\w*)\b)",
    r"(?=.*\b(incredulous\w*)\b)",
    r"(?=.*\b(concern\w*)\b)",
    r"(?=.*\b(pessimism\w*)\b)",
    r"(?=.*\b(skepticism\w*)\b)",
    r"(?=.*\b(dubiety\w*)\b)",
    r"(?=.*\b(apprehensiveness\w*)\b)",
    r"(?=.*\b(reservation\w*)\b)",
    r"(?=.*\b(untrustworthiness\w*)\b)",
    r"(?=.*\b(cautiousness\w*)\b)",
    r"(?=.*\b(hesitan\w*)\b)",
    r"(?=.*\b(surveil\w*)\b)",
    r"(?=.*\b(discredit\w*)\b)",
    r"(?=.*\b(wariness\w*)\b)",
    r"(?=.*\b(skeptical\w*)\b)",
    r"(?=.*\b(distrustful\w*)\b)",
    r"(?=.*\b(critical\w*)\b)",
    r"(?=.*\b(secrecy\w*)\b)",
    r"(?=.*\b(misgiving\w*)\b)",
    r"(?=.*\b(scrutiny\w*)\b)",
    r"(?=.*\b(apprehensive\w*)\b)"
]

## Testing zone

In [ ]:
def test_regex_filter(regex):
  """
  This function tests whether a user-provided piece of text will pass the specified regex filter

  """
  input_string = input("Sentence: ")

  for expression in regex:
    pattern = re.compile(expression)
    if pattern.match(input_string) is not None:
      return True

  return False

In [135]:
test_regex_filter(vaccine_filter)

Sentence: have covid vaccine


True

# Apply filters and note results

## function to apply filter

In [ ]:
# function to apply filter to a dataframe and return filter results

def apply_regex_filter(df, regex, new_col_name):
  df[new_col_name] = False

  for expression in regex:
    # Use 'str.contains()' to check if each tweet matches the pattern
    mask = df['text'].str.contains(expression, flags=re.IGNORECASE, regex=True)

    # Update 'no_vax_filter' column where the tweet matches the pattern
    df.loc[mask, new_col_name] = True

  return df

## apply filters to dataset

In [ ]:
random_sample = apply_regex_filter(random_sample, regex = vaccine_filter, new_col_name = "vaccine_filter")
random_sample = apply_regex_filter(random_sample, regex = hesitancy_filter, new_col_name = "hesitancy_filter")
random_sample = apply_regex_filter(random_sample, regex = mistrust_filter, new_col_name = "mistrust_filter")
random_sample = apply_regex_filter(random_sample, regex = safety_filter, new_col_name = "safety_filter")

<ipython-input-114-ca3f50699e99>:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df['text'].str.contains(expression, flags=re.IGNORECASE, regex=True)
<ipython-input-114-ca3f50699e99>:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df['text'].str.contains(expression, flags=re.IGNORECASE, regex=True)
<ipython-input-114-ca3f50699e99>:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df['text'].str.contains(expression, flags=re.IGNORECASE, regex=True)


# Explore filter performance

## define functions to explore filter performance

In [ ]:
def report_label_spread(df, filter_name):
  positive_labels = len(df[df[filter_name] == True])
  negative_labels = len(df[df[filter_name] == False])
  total_labels = len(df)

  print(f"Proportion positive labels: {positive_labels / total_labels}")
  print(f"Proportion negative labels: {negative_labels / total_labels}")


def show_positive_labels(df, filter_name):
  return df[df[filter_name] == True].loc[:, ['text', filter_name]]

## explore filter performance

In [ ]:
filter_names = ['vaccine_filter', 'hesitancy_filter', 'safety_filter', 'mistrust_filter']

for filter_name in filter_names:
  print(f"\n{filter_name}:")
  report_label_spread(df = random_sample, filter_name = filter_name)

show_positive_labels(df = random_sample, filter_name = "mistrust_filter")


vaccine_filter:
Proportion positive labels: 0.4672
Proportion negative labels: 0.5328

hesitancy_filter:
Proportion positive labels: 0.0164
Proportion negative labels: 0.9836

safety_filter:
Proportion positive labels: 0.0598
Proportion negative labels: 0.9402

mistrust_filter:
Proportion positive labels: 0.0294
Proportion negative labels: 0.9706


,text,mistrust_filter
7126,Matt Damon on COVID vaccine hesitancy: Trust s...,True
6981,"@EastEndCommuni1 I think you mean “hell, no”. ...",True
44355,#CovidVaccine !! When? How? Where? Lets answer...,True
98780,From the donkeys mouth. This then begs the que...,True
30095,This event is happening tomorrow. Get your que...,True
...,...,...
62479,The second COVID-19 vaccine dose is critical f...,True
85217,@donlemon @KizzyPhD @AOC @NIH #drfauci I ...,True
45426,Join @ourhealthiersel for a special #Instagram...,True
2290,#COVIDVaccine AotearoA \nIn New Zealand’s stat...,True


# Apply filters to dataset

## code to run cascading filters

In [202]:
def cascading_regex_filter(df, regex_filter_chain, filter_names):
  current_tweet_ids = df['tweet_id'].tolist()

  # go through each filter level in the cascade
  for filter_level in range(len(regex_filter_chain)):

    # when multiple filters occur on the same level
    if type(filter_names[filter_level]) == list:

      # apply each filter to the current set of tweets
      for filter in range(len(filter_names[filter_level])):

        # initialise filter output columns
        df[filter_names[filter_level][filter]] = False

        for tweet_id in current_tweet_ids:
          # check if the tweet id passes the regex filter
          for expression in regex_filter_chain[filter_level][filter]:
            pattern = re.compile(expression)
            if pattern.match(str(df.loc[df["tweet_id"] == tweet_id, 'text'].values[0])) is not None:
              # if it passes the filter, update the dataframe
              df.loc[df["tweet_id"] == tweet_id, filter_names[filter_level][filter]] = True
              break


    # otherwise . . .
    else:
      # initialise filter output columns
      df[filter_names[filter_level]] = False

      # go through the current list of tweet ids
      working_tweet_ids = current_tweet_ids[:]

      for tweet_id in working_tweet_ids:
        pass_filter = False
        for expression in regex_filter_chain[filter_level]:
          pattern = re.compile(expression)
          if pattern.match(str(df.loc[df["tweet_id"] == tweet_id, 'text'].values[0])) is not None:
            # if it passes the filter, update the dataframe
            df.loc[df["tweet_id"] == tweet_id, filter_names[filter_level]] = True
            pass_filter = True
            break

        # if it doesn't pass, remove that tweet id from the working tweet ids
        if pass_filter == False:
          current_tweet_ids.remove(tweet_id)

  return df

## run cascading filter on dataframe

In [203]:
regex_filter_chain = [vaccine_filter, hesitancy_filter, [safety_filter, mistrust_filter]]
filter_names = ['vaccine_filter', 'hesitancy_filter', ['safety_filter', 'mistrust_filter']]
new_random_sample = random_sample.iloc[0:5, :].reset_index().copy()

output_df = cascading_regex_filter(random_sample, regex_filter_chain, filter_names)

## explore results after cascading filters

In [204]:
filter_names = ['vaccine_filter', 'hesitancy_filter', 'safety_filter', 'mistrust_filter']

for filter_name in filter_names:
  print(f"\n{filter_name}:")
  report_label_spread(df = output_df, filter_name = filter_name)


vaccine_filter:
Proportion positive labels: 0.2799
Proportion negative labels: 0.7201

hesitancy_filter:
Proportion positive labels: 0.0101
Proportion negative labels: 0.9899

safety_filter:
Proportion positive labels: 0.0005
Proportion negative labels: 0.9995

mistrust_filter:
Proportion positive labels: 0.0049
Proportion negative labels: 0.9951


In [ ]:
from google.colab import files

output_df.to_csv('cascading_regex_filters_results.csv', encoding = 'utf-8-sig')
files.download('cascading_regex_filters_results.csv')